In [1]:
from transformers import BertTokenizer
from transformers import MarianMTModel, MarianTokenizer
import nltk
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from rich import print
from rich.table import Table
from rich.console import Console
import shutil
import torch
import sacremoses
import sentencepiece

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
terminal_width = shutil.get_terminal_size().columns

c:\Users\prith\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Data Information & Representation**

Informasi terkait dataset yang dijelaskan melalui:

<li>Null Handling</li>
<li>Cek Outlier</li>
<li>Split En & De</li>
<li>?</li>


In [ ]:
data = pd.read_csv("multilang-ticket.csv")
tiket_df = pd.DataFrame(data)
console = Console(width=terminal_width)
console.rule("[bold white]Data Information & Representation")
print(tiket_df.head())
console.rule("[bold white]Data Info")
print("data info:")
print(tiket_df.info(5))
numeric_columns = tiket_df.select_dtypes(include=[np.number]).columns
categorical_columns = tiket_df.select_dtypes(include=["string", "object"]).columns
print(f"Data numerik mencakup: {numeric_columns.to_list()}")
print(f"Data kategorikal mencakup: {categorical_columns.to_list()}")

console.rule("[bold white]Null Handling")
print(tiket_df.isnull().sum())

console.rule("[bold white]Split Data Bahasa Inggris")
en_df = tiket_df[tiket_df["language"] == "en"]
ed_df = pd.DataFrame(en_df)
print(en_df["body"].head(5))
print(en_df["body"].count())
console.rule("[bold white]Split Data Bahasa Belanda")
de_df = tiket_df[tiket_df["language"] == "de"]
print(de_df["body"].head(5))
print(de_df["body"].count())

────────────────────── Data Information & Representation ───────────────────────

subject  \
0  Unvorhergesehener Absturz der Datenanalyse-Pla...   
1                           Customer Support Inquiry   
2                      Data Analytics for Investment   
3                 Krankenhaus-Dienstleistung-Problem   
4                                           Security   

                                                body  \
0  Die Datenanalyse-Plattform brach unerwartet ab...   
1  Seeking information on digital strategies that...   
2  I am contacting you to request information on ...   
3  Ein Medien-Daten-Sperrverhalten trat aufgrund ...   
4  Dear Customer Support, I am reaching out to in...   

                                              answer      type  \
0  Ich werde Ihnen bei der Lösung des Problems he...  Incident   
1  We offer a variety of digital strategies and s...   Request   
2  I am here to assist you with data analytics to...   Request   
3  Zurück zur E-Mail-Beschwerde über den Sperrver...  Incident   
4  Dear [name], we take the security of medical d...   Request   

              queue priority language      tag_1      tag_2       tag_3  \
0   General Inquiry      low       de      Crash  Technical         Bug   
1  Customer Service   medium       en   Feedback      Sales          IT   
2  Customer Service   medium       en  Technical    Product    Guidance   
3  Customer Service     high       de   Security     Breach       Login   
4  Customer Service   medium       en   Security   Customer  Compliance   

           tag_4          tag_5       tag_6          tag_7 tag_8  
0       Hardware     Resolution      Outage  Documentation   NaN  
1   Tech Support            NaN         NaN            NaN   NaN  
2  Documentation    Performance     Feature            NaN   NaN  
3    Maintenance       Incident  Resolution       Feedback   NaN  
4         Breach  Documentation    Guidance            NaN   NaN

────────────────────────────────── Data Info ───────────────────────────────────

data info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   18539 non-null  object
 1   body      19998 non-null  object
 2   answer    19996 non-null  object
 3   type      20000 non-null  object
 4   queue     20000 non-null  object
 5   priority  20000 non-null  object
 6   language  20000 non-null  object
 7   tag_1     20000 non-null  object
 8   tag_2     19954 non-null  object
 9   tag_3     19905 non-null  object
 10  tag_4     18461 non-null  object
 11  tag_5     13091 non-null  object
 12  tag_6     7351 non-null   object
 13  tag_7     3928 non-null   object
 14  tag_8     1907 non-null   object
dtypes: object(15)
memory usage: 2.3+ MB


...

1    Seeking information on digital strategies that...
2    I am contacting you to request information on ...
4    Dear Customer Support, I am reaching out to in...
5    Inquiring about best practices for securing me...
7    The integration stopped working unexpectedly, ...
Name: body, dtype: object

11922

────────────────────────── Split Data Bahasa Belanda ───────────────────────────

0    Die Datenanalyse-Plattform brach unerwartet ab...
3    Ein Medien-Daten-Sperrverhalten trat aufgrund ...
6    Ratung, ob es möglich ist, Sicherung medizinis...
8    Dear Customer Support, I am writing in regards...
9    The latest data analysis reports are inconsist...
Name: body, dtype: object

8076

In [ ]:
model_name = "Helsinki-NLP/opus-mt-de-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
de_df = pd.DataFrame(de_df["body"].head(7))
print(de_df)


def translate_german_to_english(text):
    if not isinstance(text, str):
        return print(f"not translated {text}")
    tokenized_text = tokenizer(text, return_tensors="pt")
    translated = model.generate(**tokenized_text)
    english_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

    return english_text


de_df["body_en"] = de_df.apply(translate_german_to_english)
print(de_df["body_en"])

body
0   Die Datenanalyse-Plattform brach unerwartet ab...
3   Ein Medien-Daten-Sperrverhalten trat aufgrund ...
6   Ratung, ob es möglich ist, Sicherung medizinis...
8   Dear Customer Support, I am writing in regards...
9   The latest data analysis reports are inconsist...
20  Sehr geehrte Kundensupport, wir begegnen momen...
25  Unverschlüsselte medizinische Daten auf ungesc...

not translated 0     Die Datenanalyse-Plattform brach unerwartet ab...
3     Ein Medien-Daten-Sperrverhalten trat aufgrund ...
6     Ratung, ob es möglich ist, Sicherung medizinis...
8     Dear Customer Support, I am writing in regards...
9     The latest data analysis reports are inconsist...
20    Sehr geehrte Kundensupport, wir begegnen momen...
25    Unverschlüsselte medizinische Daten auf ungesc...
Name: body, dtype: object

0     NaN
3     NaN
6     NaN
8     NaN
9     NaN
20    NaN
25    NaN
Name: body_en, dtype: object